# 📅 Gestor Interactivo de Calendario

Este notebook te permite crear, editar, eliminar y visualizar eventos de calendario de manera interactiva.

## Características:
- ✅ Crear nuevos eventos
- ✏️ Editar eventos existentes
- 🗑️ Eliminar eventos
- 📊 Visualizar calendario mensual
- 💾 Guardar y cargar eventos desde archivo

In [1]:
# Instalar dependencias necesarias
import sys
!{sys.executable} -m pip install ipywidgets pandas

  Using cached ipywidgets-8.1.7-py3-none-any.whl.metadata (2.4 kB)
  Using cached widgetsnbextension-4.0.14-py3-none-any.whl.metadata (1.6 kB)
  Using cached jupyterlab_widgets-3.0.15-py3-none-any.whl.metadata (20 kB)
Using cached ipywidgets-8.1.7-py3-none-any.whl (139 kB)
Using cached jupyterlab_widgets-3.0.15-py3-none-any.whl (216 kB)
Using cached widgetsnbextension-4.0.14-py3-none-any.whl (2.2 MB)
   ---------------------------------------- 0.0/11.3 MB ? eta -:--:--
   ---------------------------- ----------- 8.1/11.3 MB 42.1 MB/s eta 0:00:01
   ---------------------------------------- 11.3/11.3 MB 41.8 MB/s  0:00:00

   ---------------------------------------- 0/6 [pytz]
   ---------------------------------------- 0/6 [pytz]
   ------------- -------------------------- 2/6 [tzdata]
   ------------- -------------------------- 2/6 [tzdata]
   -------------------------- ------------- 4/6 [pandas]
   -------------------------- ------------- 4/6 [pandas]
   -------------------------- ---


[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
# Importar librerías necesarias
import json
import os
from datetime import datetime, timedelta
from pathlib import Path
import ipywidgets as widgets
from IPython.display import display, clear_output, HTML
import pandas as pd

In [3]:
# Clase para gestionar eventos del calendario
class CalendarioManager:
    def __init__(self, archivo='eventos.json'):
        self.archivo = archivo
        self.eventos = self.cargar_eventos()
    
    def cargar_eventos(self):
        """Cargar eventos desde archivo JSON"""
        if os.path.exists(self.archivo):
            try:
                with open(self.archivo, 'r', encoding='utf-8') as f:
                    return json.load(f)
            except:
                return []
        return []
    
    def guardar_eventos(self):
        """Guardar eventos en archivo JSON"""
        with open(self.archivo, 'w', encoding='utf-8') as f:
            json.dump(self.eventos, f, indent=2, ensure_ascii=False)
    
    def agregar_evento(self, titulo, fecha, hora_inicio, hora_fin, descripcion='', categoria='General'):
        """Agregar un nuevo evento"""
        evento = {
            'id': len(self.eventos) + 1,
            'titulo': titulo,
            'fecha': fecha,
            'hora_inicio': hora_inicio,
            'hora_fin': hora_fin,
            'descripcion': descripcion,
            'categoria': categoria,
            'creado': datetime.now().strftime('%Y-%m-%d %H:%M:%S')
        }
        self.eventos.append(evento)
        self.guardar_eventos()
        return evento
    
    def editar_evento(self, evento_id, **kwargs):
        """Editar un evento existente"""
        for evento in self.eventos:
            if evento['id'] == evento_id:
                evento.update(kwargs)
                evento['modificado'] = datetime.now().strftime('%Y-%m-%d %H:%M:%S')
                self.guardar_eventos()
                return evento
        return None
    
    def eliminar_evento(self, evento_id):
        """Eliminar un evento"""
        self.eventos = [e for e in self.eventos if e['id'] != evento_id]
        self.guardar_eventos()
    
    def obtener_eventos_por_fecha(self, fecha):
        """Obtener eventos de una fecha específica"""
        return [e for e in self.eventos if e['fecha'] == fecha]
    
    def obtener_eventos_del_mes(self, año, mes):
        """Obtener eventos de un mes específico"""
        eventos_mes = []
        for e in self.eventos:
            fecha_evento = datetime.strptime(e['fecha'], '%Y-%m-%d')
            if fecha_evento.year == año and fecha_evento.month == mes:
                eventos_mes.append(e)
        return eventos_mes
    
    def obtener_evento_por_id(self, evento_id):
        """Obtener un evento por su ID"""
        for evento in self.eventos:
            if evento['id'] == evento_id:
                return evento
        return None

# Inicializar el gestor de calendario
calendario = CalendarioManager()
print(f"✅ Calendario inicializado. Total de eventos: {len(calendario.eventos)}")

✅ Calendario inicializado. Total de eventos: 0


In [4]:
# Interfaz para crear nuevos eventos
def crear_interfaz_nuevo_evento():
    output = widgets.Output()
    
    # Widgets de entrada
    titulo_input = widgets.Text(
        description='Título:',
        placeholder='Ej: Reunión importante',
        style={'description_width': '120px'},
        layout=widgets.Layout(width='500px')
    )
    
    fecha_input = widgets.DatePicker(
        description='Fecha:',
        value=datetime.now().date(),
        style={'description_width': '120px'}
    )
    
    hora_inicio_input = widgets.Text(
        description='Hora inicio:',
        placeholder='HH:MM (Ej: 09:00)',
        value='09:00',
        style={'description_width': '120px'},
        layout=widgets.Layout(width='300px')
    )
    
    hora_fin_input = widgets.Text(
        description='Hora fin:',
        placeholder='HH:MM (Ej: 10:00)',
        value='10:00',
        style={'description_width': '120px'},
        layout=widgets.Layout(width='300px')
    )
    
    categoria_input = widgets.Dropdown(
        options=['General', 'Trabajo', 'Personal', 'Reunión', 'Cumpleaños', 'Otro'],
        value='General',
        description='Categoría:',
        style={'description_width': '120px'}
    )
    
    descripcion_input = widgets.Textarea(
        description='Descripción:',
        placeholder='Descripción del evento (opcional)',
        style={'description_width': '120px'},
        layout=widgets.Layout(width='500px', height='100px')
    )
    
    boton_crear = widgets.Button(
        description='✅ Crear Evento',
        button_style='success',
        icon='check'
    )
    
    def on_crear_click(b):
        with output:
            clear_output()
            if not titulo_input.value:
                print('❌ Error: El título es obligatorio')
                return
            
            fecha_str = fecha_input.value.strftime('%Y-%m-%d')
            
            evento = calendario.agregar_evento(
                titulo=titulo_input.value,
                fecha=fecha_str,
                hora_inicio=hora_inicio_input.value,
                hora_fin=hora_fin_input.value,
                descripcion=descripcion_input.value,
                categoria=categoria_input.value
            )
            
            print(f'✅ Evento creado exitosamente!')
            print(f'ID: {evento["id"]}')
            print(f'Título: {evento["titulo"]}')
            print(f'Fecha: {evento["fecha"]}')
            print(f'Horario: {evento["hora_inicio"]} - {evento["hora_fin"]}')
            
            # Limpiar campos
            titulo_input.value = ''
            descripcion_input.value = ''
    
    boton_crear.on_click(on_crear_click)
    
    # Mostrar interfaz
    display(HTML('<h3>➕ Crear Nuevo Evento</h3>'))
    display(widgets.VBox([
        titulo_input,
        fecha_input,
        hora_inicio_input,
        hora_fin_input,
        categoria_input,
        descripcion_input,
        boton_crear,
        output
    ]))

crear_interfaz_nuevo_evento()

In [ ]:
# Visualizar todos los eventos
def mostrar_todos_eventos():
    if not calendario.eventos:
        print('📭 No hay eventos registrados')
        return
    
    # Crear DataFrame para mejor visualización
    df = pd.DataFrame(calendario.eventos)
    df = df[['id', 'titulo', 'fecha', 'hora_inicio', 'hora_fin', 'categoria']]
    df = df.sort_values('fecha')
    
    display(HTML(f'<h3>📋 Lista de Eventos ({len(calendario.eventos)} total)</h3>'))
    display(df.to_html(index=False))

mostrar_todos_eventos()

In [ ]:
# Interfaz para editar eventos
def crear_interfaz_editar_evento():
    output = widgets.Output()
    
    # Selector de evento
    eventos_opciones = {f"ID {e['id']}: {e['titulo']} ({e['fecha']})": e['id'] for e in calendario.eventos}
    
    if not eventos_opciones:
        print('📭 No hay eventos para editar')
        return
    
    evento_selector = widgets.Dropdown(
        options=eventos_opciones,
        description='Seleccionar:',
        style={'description_width': '120px'},
        layout=widgets.Layout(width='500px')
    )
    
    # Campos de edición
    titulo_edit = widgets.Text(description='Título:', style={'description_width': '120px'}, layout=widgets.Layout(width='500px'))
    fecha_edit = widgets.DatePicker(description='Fecha:', style={'description_width': '120px'})
    hora_inicio_edit = widgets.Text(description='Hora inicio:', style={'description_width': '120px'}, layout=widgets.Layout(width='300px'))
    hora_fin_edit = widgets.Text(description='Hora fin:', style={'description_width': '120px'}, layout=widgets.Layout(width='300px'))
    categoria_edit = widgets.Dropdown(
        options=['General', 'Trabajo', 'Personal', 'Reunión', 'Cumpleaños', 'Otro'],
        description='Categoría:',
        style={'description_width': '120px'}
    )
    descripcion_edit = widgets.Textarea(description='Descripción:', style={'description_width': '120px'}, layout=widgets.Layout(width='500px', height='100px'))
    
    def cargar_evento(change):
        evento_id = evento_selector.value
        evento = calendario.obtener_evento_por_id(evento_id)
        if evento:
            titulo_edit.value = evento['titulo']
            fecha_edit.value = datetime.strptime(evento['fecha'], '%Y-%m-%d').date()
            hora_inicio_edit.value = evento['hora_inicio']
            hora_fin_edit.value = evento['hora_fin']
            categoria_edit.value = evento['categoria']
            descripcion_edit.value = evento.get('descripcion', '')
    
    evento_selector.observe(cargar_evento, names='value')
    
    boton_guardar = widgets.Button(
        description='💾 Guardar Cambios',
        button_style='info',
        icon='save'
    )
    
    def on_guardar_click(b):
        with output:
            clear_output()
            evento_id = evento_selector.value
            fecha_str = fecha_edit.value.strftime('%Y-%m-%d')
            
            calendario.editar_evento(
                evento_id,
                titulo=titulo_edit.value,
                fecha=fecha_str,
                hora_inicio=hora_inicio_edit.value,
                hora_fin=hora_fin_edit.value,
                categoria=categoria_edit.value,
                descripcion=descripcion_edit.value
            )
            
            print(f'✅ Evento ID {evento_id} actualizado exitosamente!')
    
    boton_guardar.on_click(on_guardar_click)
    
    # Cargar primer evento
    cargar_evento(None)
    
    display(HTML('<h3>✏️ Editar Evento</h3>'))
    display(widgets.VBox([
        evento_selector,
        titulo_edit,
        fecha_edit,
        hora_inicio_edit,
        hora_fin_edit,
        categoria_edit,
        descripcion_edit,
        boton_guardar,
        output
    ]))

crear_interfaz_editar_evento()

In [ ]:
# Interfaz para eliminar eventos
def crear_interfaz_eliminar_evento():
    output = widgets.Output()
    
    eventos_opciones = {f"ID {e['id']}: {e['titulo']} ({e['fecha']})": e['id'] for e in calendario.eventos}
    
    if not eventos_opciones:
        print('📭 No hay eventos para eliminar')
        return
    
    evento_selector = widgets.Dropdown(
        options=eventos_opciones,
        description='Seleccionar:',
        style={'description_width': '120px'},
        layout=widgets.Layout(width='500px')
    )
    
    boton_eliminar = widgets.Button(
        description='🗑️ Eliminar Evento',
        button_style='danger',
        icon='trash'
    )
    
    def on_eliminar_click(b):
        with output:
            clear_output()
            evento_id = evento_selector.value
            evento = calendario.obtener_evento_por_id(evento_id)
            
            if evento:
                print(f'🗑️ Eliminando evento: {evento["titulo"]}')
                calendario.eliminar_evento(evento_id)
                print(f'✅ Evento ID {evento_id} eliminado exitosamente!')
                print('⚠️ Ejecuta de nuevo esta celda para actualizar la lista')
    
    boton_eliminar.on_click(on_eliminar_click)
    
    display(HTML('<h3>🗑️ Eliminar Evento</h3>'))
    display(widgets.VBox([
        evento_selector,
        boton_eliminar,
        output
    ]))

crear_interfaz_eliminar_evento()

In [ ]:
# Visualizar calendario mensual
def visualizar_calendario_mes():
    output = widgets.Output()
    
    # Selector de mes y año
    mes_selector = widgets.Dropdown(
        options=[
            ('Enero', 1), ('Febrero', 2), ('Marzo', 3), ('Abril', 4),
            ('Mayo', 5), ('Junio', 6), ('Julio', 7), ('Agosto', 8),
            ('Septiembre', 9), ('Octubre', 10), ('Noviembre', 11), ('Diciembre', 12)
        ],
        value=datetime.now().month,
        description='Mes:'
    )
    
    año_selector = widgets.IntText(
        value=datetime.now().year,
        description='Año:'
    )
    
    boton_mostrar = widgets.Button(
        description='📅 Mostrar Calendario',
        button_style='primary'
    )
    
    def on_mostrar_click(b):
        with output:
            clear_output()
            mes = mes_selector.value
            año = año_selector.value
            
            eventos_mes = calendario.obtener_eventos_del_mes(año, mes)
            
            meses_nombres = ['', 'Enero', 'Febrero', 'Marzo', 'Abril', 'Mayo', 'Junio',
                           'Julio', 'Agosto', 'Septiembre', 'Octubre', 'Noviembre', 'Diciembre']
            
            print(f'\n📅 Calendario de {meses_nombres[mes]} {año}')
            print('=' * 80)
            print(f'\nTotal de eventos en este mes: {len(eventos_mes)}\n')
            
            if eventos_mes:
                # Agrupar por fecha
                eventos_por_fecha = {}
                for evento in eventos_mes:
                    fecha = evento['fecha']
                    if fecha not in eventos_por_fecha:
                        eventos_por_fecha[fecha] = []
                    eventos_por_fecha[fecha].append(evento)
                
                for fecha in sorted(eventos_por_fecha.keys()):
                    fecha_obj = datetime.strptime(fecha, '%Y-%m-%d')
                    dias_semana = ['Lun', 'Mar', 'Mié', 'Jue', 'Vie', 'Sáb', 'Dom']
                    dia_semana = dias_semana[fecha_obj.weekday()]
                    
                    print(f'\n📆 {dia_semana} {fecha_obj.day:02d}/{mes:02d}/{año}')
                    print('-' * 80)
                    
                    for evento in eventos_por_fecha[fecha]:
                        print(f"  🔹 [{evento['categoria']}] {evento['titulo']}")
                        print(f"     ⏰ {evento['hora_inicio']} - {evento['hora_fin']}")
                        if evento.get('descripcion'):
                            print(f"     📝 {evento['descripcion']}")
                        print()
            else:
                print('📭 No hay eventos registrados para este mes')
    
    boton_mostrar.on_click(on_mostrar_click)
    
    display(HTML('<h3>📅 Vista de Calendario Mensual</h3>'))
    display(widgets.HBox([mes_selector, año_selector]))
    display(boton_mostrar)
    display(output)

visualizar_calendario_mes()

In [ ]:
# Buscar eventos por fecha específica
def buscar_eventos_por_fecha():
    output = widgets.Output()
    
    fecha_input = widgets.DatePicker(
        description='Fecha:',
        value=datetime.now().date()
    )
    
    boton_buscar = widgets.Button(
        description='🔍 Buscar',
        button_style='primary'
    )
    
    def on_buscar_click(b):
        with output:
            clear_output()
            fecha_str = fecha_input.value.strftime('%Y-%m-%d')
            eventos = calendario.obtener_eventos_por_fecha(fecha_str)
            
            print(f'\n🔍 Eventos para {fecha_str}:')
            print('=' * 80)
            
            if eventos:
                for evento in eventos:
                    print(f"\n🔹 {evento['titulo']}")
                    print(f"   ID: {evento['id']}")
                    print(f"   ⏰ Horario: {evento['hora_inicio']} - {evento['hora_fin']}")
                    print(f"   📁 Categoría: {evento['categoria']}")
                    if evento.get('descripcion'):
                        print(f"   📝 Descripción: {evento['descripcion']}")
                print(f'\n✅ Total: {len(eventos)} evento(s)')
            else:
                print('\n📭 No hay eventos para esta fecha')
    
    boton_buscar.on_click(on_buscar_click)
    
    display(HTML('<h3>🔍 Buscar Eventos por Fecha</h3>'))
    display(widgets.HBox([fecha_input, boton_buscar]))
    display(output)

buscar_eventos_por_fecha()

In [ ]:
# Estadísticas del calendario
def mostrar_estadisticas():
    print('\n📊 Estadísticas del Calendario')
    print('=' * 80)
    
    total_eventos = len(calendario.eventos)
    print(f'\n📌 Total de eventos: {total_eventos}')
    
    if total_eventos > 0:
        # Eventos por categoría
        categorias = {}
        for evento in calendario.eventos:
            cat = evento['categoria']
            categorias[cat] = categorias.get(cat, 0) + 1
        
        print('\n📁 Eventos por categoría:')
        for cat, count in sorted(categorias.items(), key=lambda x: x[1], reverse=True):
            print(f'   {cat}: {count}')
        
        # Próximos eventos
        hoy = datetime.now().date()
        proximos = []
        pasados = []
        
        for evento in calendario.eventos:
            fecha_evento = datetime.strptime(evento['fecha'], '%Y-%m-%d').date()
            if fecha_evento >= hoy:
                proximos.append(evento)
            else:
                pasados.append(evento)
        
        print(f'\n📅 Eventos próximos: {len(proximos)}')
        print(f'📜 Eventos pasados: {len(pasados)}')
        
        if proximos:
            proximos.sort(key=lambda x: x['fecha'])
            print('\n🔜 Próximos 5 eventos:')
            for evento in proximos[:5]:
                print(f"   • {evento['fecha']} - {evento['titulo']} ({evento['hora_inicio']})")
    else:
        print('\n📭 No hay eventos registrados aún')

mostrar_estadisticas()

## 💡 Instrucciones de Uso

1. **Crear eventos**: Usa la celda "Crear Nuevo Evento" para agregar eventos al calendario
2. **Ver eventos**: La celda "Lista de Eventos" muestra todos los eventos en formato tabla
3. **Editar eventos**: Usa la celda "Editar Evento" para modificar eventos existentes
4. **Eliminar eventos**: Usa la celda "Eliminar Evento" para borrar eventos
5. **Vista mensual**: La celda "Vista de Calendario Mensual" muestra todos los eventos de un mes
6. **Buscar**: Usa la celda "Buscar Eventos por Fecha" para encontrar eventos de un día específico
7. **Estadísticas**: Revisa las estadísticas generales de tu calendario

Los eventos se guardan automáticamente en el archivo `eventos.json` en la misma carpeta del notebook.

In [ ]:
# Recargar eventos (útil si modificaste el archivo JSON manualmente)
calendario.eventos = calendario.cargar_eventos()
print(f'✅ Eventos recargados. Total: {len(calendario.eventos)}')